In [1]:
from pickle import load
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords
from sklearn.pipeline import make_pipeline
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import cross_validate
from sklearn.metrics import make_scorer, fbeta_score
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score, make_scorer, f1_score, precision_score, recall_score
from sklearn import svm
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline as imbpipe
from sklearn.ensemble import RandomForestClassifier

In [2]:
df = load(open('df_clean.pkl', 'rb'))

In [3]:
df.head()

,text,company,target,product
0,wesley83 i have a 3g iphone after 3 hrs tweeti...,apple,0,iphone
1,jessedee know about fludapp awesome ipadiphon...,apple,1,ipad or iphone app
2,swonderlin can not wait for ipad 2 also they s...,apple,1,ipad
3,sxsw i hope this years festival isnt as crashy...,apple,0,ipad or iphone app
4,sxtxstate great stuff on fri sxsw marissa maye...,google,1,google


In [4]:
X = df['text']
y = df.target

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify = y)

In [6]:
sw = stopwords.words('english')

In [7]:
# oversample = SMOTE()
# X_train, y_train = oversample.fit_resample(X_train, y_train)

In [8]:
micro_recall = make_scorer(recall_score, average = 'micro')

In [9]:
f_1_weighted = make_scorer(f1_score, average = 'weighted')

In [10]:
# dt_pipe = imbpipe(steps=[
#     ('ss', StandardScaler()),
#     ('sm', SMOTE(random_state=42)),
#     ('dtc', DecisionTreeClassifier(random_state=42, 
#                                    max_depth= 6, 
#                                    class_weight = 'balanced'))])

In [12]:
pipe = imbpipe(steps = [
    ('tfid', TfidfVectorizer(max_features=300, stop_words= sw)),
    ('sm', SMOTE()),
    ('rfc', RandomForestClassifier(class_weight = 'balanced'))])


In [13]:
cv = cross_validate(pipe, X_train, y_train, scoring = micro_recall)
cv

{'fit_time': array([10.48226213, 10.8861444 , 11.0652349 , 11.2754848 , 10.69470835]),
 'score_time': array([0.29121947, 0.08975887, 0.24003911, 0.10124826, 0.29571104]),
 'test_score': array([0.61747254, 0.58777261, 0.60156439, 0.60707768, 0.59298706])}

In [14]:
y_hat_train = cross_val_predict(pipe, X_train, y_train)

In [15]:
confusion_matrix(y_train, y_hat_train)

array([[  88,  109,  228],
       [  52, 1082, 1084],
       [ 123,  801, 2518]], dtype=int64)